In [ ]:
# Création dataset infos films
import pandas as pd

df_basics=pd.read_csv('../../dataset/title.basics.tsv', sep='\t')
df_basics.head(20)
df_crew=pd.read_csv('../../dataset/title.crew.tsv', sep='\t')
df_name=pd.read_csv('../../dataset/name.basics.tsv', sep='\t')
df_principals=pd.read_csv('../../dataset/title.principals.tsv', sep='\t')
df_ratings=pd.read_csv('../../dataset/title.ratings.tsv', sep='\t')

In [ ]:
#df_basics.groupby("titleType").count()
# 514026 films
df_basics.head()

In [ ]:
#df_crew.head(30)
print(df_name.head())
print(df_crew.head())
df_principals.head()

In [ ]:
from tqdm import tqdm 

#Rajout à df_basics des colonnes discrétisés par genre

genres_films=['action', 'adult', 'adventure', 'animation', 'biography','comedy','crime', 'documentary', 
               'drama', 'family','fantasy', 'game-show','history', 'horror', 'romance','sci-fi', 'short', 
              'sport','talk-show', 'thriller','war', 'western']
for element in tqdm(genres_films):
    df_basics[element]=df_basics["genres"].map(lambda x: int(element in str(x).lower()))


In [5]:
#Etude des genres de film. certains genres sont minoritaires, 
#et ne se retrouvent pas dans base de donnée movieLens
#gameshow 189 091
#talk-show 535 391
#short 766 586
#sport 119 865

In [6]:
#tri des données pour ne garder que les films long-métrage
df=df_basics.drop(df_basics[df_basics["talk-show"]==1].index)
df=df.drop(df[df["game-show"]==1].index)
df=df.drop(df[df["short"]==1].index)
df=df.drop(df[df["sport"]==1].index)
df=df.drop(df[df["titleType"]!="movie"].index)

In [7]:
#Suppression des colonnes inutiles
df=df.drop(["talk-show", "sport", "short", "game-show", "titleType", "endYear", "genres"], axis=1)

In [8]:
#Ajout des colonnes directors et writers
df_crew.index=df_crew.tconst
df_crew=df_crew.drop(["tconst"], axis=1)
df=df.join(df_crew, on=df.tconst)

In [9]:
#Ajout de la colonne "Actors" avec liste des acteurs 
actor=df_principals[df_principals.category == "actor"]
actors=pd.Series(actor.groupby("tconst")['nconst'].apply(list), name="actors")
df=df.join(actors, on="tconst")

In [10]:
#Ajout des colonnes ratings mean et nb_votes
df_ratings.index=df_ratings.tconst
df_ratings=df_ratings.drop(["tconst"], axis=1)
df=df.join(df_ratings, on=df.tconst)

In [11]:
#Transformation df.tconst str en int
import re

serie=pd.Series(df.tconst)
imdb_id=[]
for element in tqdm(serie):
    imdb_id.append(int(re.search(r'\d+', element).group()))

imdb_id_2=[]
for element in imdb_id:
    imdb_id_2.append(int(str(element)))
df.index=imdb_id_2
df.head()

100%|██████████| 509078/509078 [00:01<00:00, 398467.71it/s]


,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,action,adult,adventure,animation,...,romance,sci-fi,thriller,war,western,directors,writers,actors,averageRating,numVotes
9,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,0,0,0,0,...,1,0,0,0,0,nm0085156,nm0085156,[nm0183823],5.6,76.0
335,tt0000335,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,0,0,0,0,...,0,0,0,0,0,"nm0675140,nm0095714",\N,"[nm1012612, nm1011210, nm1012621, nm0675239, n...",6.3,39.0
502,tt0000502,Bohemios,Bohemios,0,1905,100,0,0,0,0,...,0,0,0,0,0,nm0063413,"nm0063413,nm0657268,nm0675388","[nm0215752, nm0252720]",NaN,NaN
574,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,0,0,0,0,...,0,0,0,0,0,nm0846879,nm0846879,"[nm0846894, nm3002376]",6.2,503.0
615,tt0000615,Robbery Under Arms,Robbery Under Arms,0,1907,\N,0,0,0,0,...,0,0,0,0,0,nm0533958,"nm0092809,nm0533958","[nm3071427, nm0581353, nm0888988, nm0240418, n...",4.8,14.0


In [12]:
#Enregistrement du nouveau fichier
df.to_csv('../new_dataset/imbd_df.csv', sep=',', columns=df.columns)